
LangChain과 ChromaDB를 이용한 간단한 검색 시스템 구축(LLM 없음)

목적:

LangChain과 ChromaDB를 활용해 문서 검색 시스템을 구축하고, 한국어 데이터세트에서 가장 관련성이 높은 문서나 텍스트 조각을 반환하여 질의에 답한다.

1. 환경 설정

필요한 라이브러리를 설치:
LangChain
Chroma DB
개발에는 Python IDE 사용 (VScode)

2. 소규모 데이터 세트 준비

100-200개 항목으로 한국어 데이터 세트를 만들거나 다운로드.
: 한국 속담과 그 의미.
( 간단한 설명과 함께 소개된 한국의 유명한 역사적 사건 목록,
인기 있는 한국 요리와 재료 )
데이터 세트를 제목과 설명이 두 개의 열로 구성되어있는 CSV 파일로 포맷한다.

3. ChromaDB에 데이터 로드

데이터 세트를 사전 처리한다.
LangChain 호환 임베딩 모델을 사용하여 설명 열을 벡터 임베딩으로 변환한다.
ChromaDB 데이터베이스에 임베딩을 저장한다.

4. 검색 파이프라인 구현

다음과 같은 쿼리 시스템을 만듭니다.
The user inputs a query in Korean (e.g., “~~~ 속담 뜻이 뭐야? ”).
시스템은 쿼리와의 유사성을 기반으로 ChromaDB 데이터베이스에서 가장 관련성 있는 설명을 검색한다.

5. 검색 결과 평가

20~30개의 쿼리로 시스템을 테스트하고 검색된 결과의 정확성을 수동으로 확인한다.
예시 쿼리:
“다음 속담의 의미를 설명하시오: 하늘이 무너져도 솟아날 구멍이 있다."

6. 성과물

검색 시스템을 구현하는 Python 스크립트
데이터 세트에 대한 설명.
검색 과정에 대한 설명.
테스트 쿼리의 결과(입력 및 검색된 설명).
직면한 과제와 실행된 해결책.

In [1]:
!pip install langchain chromadb

## **1) 필요한 라이브러리 설치**

---

```
LangChain과 ChromaDB를 설치합니다.
```


In [2]:
import pandas as pd
import langchain
import chromadb
import os
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

os.environ["OPENAI_API_KEY"] = "..."

## **2) 소규모 데이터 세트 준비**

---

200개 항목으로 구성된 한국어 데이터 세트를 CSV 파일로 포맷한다.
속담과 관용구로 데이터 세트를 제목과 설명 두 개의 열로 구성되어있다.

In [3]:
# CSV 파일 로드 및 문서 분리
# CSVLoader를 사용하여 지정된 파일 경로에서 데이터를 로드합니다.
# 'cp949' 인코딩을 사용하여 한글 데이터가 깨지지 않도록 처리합니다.
# 로드된 데이터를 documents 변수에 저장합니다.
csv_loader = CSVLoader(file_path='/content/nanolabdataset.csv', encoding='cp949')
documents = csv_loader.load()

# 텍스트 분리기 초기화
# CharacterTextSplitter는 텍스트를 일정 크기로 분리하는 도구입니다.
# 'chunk_size=100'은 각 텍스트 조각의 최대 길이를 100자로 설정합니다.
# 'chunk_overlap=0'은 텍스트 조각 간 중첩이 없도록 설정합니다.
text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=0)

# 문서 분리 실행
# 로드된 문서 데이터를 100자 크기로 분리하여 검색에 적합한 형식으로 변환합니다.
texts = text_splitter.split_documents(documents)

RuntimeError: Error loading /content/nanolabdataset.csv

## **3) ChromaDB에 데이터를 로드합니다**

---

원본 데이터를 검색 시스템에 적합하도록 전처리한다.

데이터는 CharacterTextSplitter를 통해 100자로 나눠져 있다.
LangChain 호환 임베딩 모델을 사용하여 설명 열을 벡터 임베딩으로 변환한다.

OpenAIEmbeddings는 LangChain에서 제공하는 호환 벡터 임베딩 모델이다. 각 텍스트를 고차원 벡터로 변환한다.

```
texts
OpenAIEmbeddings
Chroma.from_documents
```
ChromaDB에 데이터 로드


In [ ]:
# 벡터 임베딩 및 ChromaDB 저장
# OpenAIEmbeddings는 LangChain에서 제공하는 임베딩 모델입니다.
# 입력된 텍스트 데이터를 고차원 벡터로 변환하여 의미론적 검색에 사용됩니다.
embeddings = OpenAIEmbeddings()

# Chroma.from_documents()는 텍스트 데이터를 벡터로 변환한 후 ChromaDB 데이터베이스에 저장합니다.
# 'texts'는 사전에 처리된 텍스트 조각이고, 'embeddings'는 이를 벡터화하는 데 사용됩니다.
# 결과로 반환된 'vector_store'는 벡터 데이터베이스로, 검색에 사용됩니다.
vector_store = Chroma.from_documents(texts, embeddings)
# 검색기 생성
retriever = vector_store.as_retriever(search_kwargs={"k": 10})  # 가장 유사한 2개 문서 검색

system_template="""Use the following pieces of context to answer the users question shortly.
Given the following summaries of a long document and a question, create a final answer with references ("SOURCES"), use "SOURCES" in capital letters regardless of the number of sources.
If you don't know the answer, just say that "I don't know", don't try to make up an answer.
----------------
{summaries}

You MUST answer in Korean:"""



## **4) 검색 파이프라인 구현**

---

데이터를 가지고 쿼리 시스템을 만듭니다.

### **쿼리 시스템**
쿼리 시스템은 사용자가 입력한 질문(쿼리)과 사전에 저장된 데이터베이스(ChromaDB)의 정보를 비교하여 가장 관련성이 높은 결과를 반환하는 기능을 구현합니다. 이 과정은 크게 사용자 입력 처리, 유사성 검색, 결과 반환 세 단계로 나뉩니다.

직접 쿼리를 입력하고 결과를 출력하는 시스템으로 구현하였습니다.




In [ ]:
# 사용자 입력 기반 검색 시스템
# 사용자가 질문을 입력하면 시스템이 관련 데이터를 검색하고 결과를 반환합니다.
while True:
    user_query = input("질문을 입력하세요: (종료하려면 '종료'를 입력하세요): ").strip()
    if user_query.lower() == '종료':
        print("검색 시스템을 종료합니다.")
        break
    # 사용자가 질문을 받고 처리한다.

    # 검색 결과 가져오기
    # 사용자의 질문(user_query)과 관련된 문서를 ChromaDB에서 검색
    results = retriever.get_relevant_documents(user_query)

    if results:
      print("\n[검색 결과]")
      # 검색된 결과 중 가장 관련성이 높은 첫 번째 결과만 출력
    # 가장 첫 번째 결과만 출력
      result = results[0]  # 첫 번째 결과 가져오기
      print(f"1. {result.page_content}")
      print("\n" + "="*50 + "\n")
    else:
      print("\n관련 문서를 찾을 수 없습니다.\n")

